<a href="https://colab.research.google.com/github/alikaiser12/AI/blob/main/Money_receipt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===========================================
# Receipt Pre-Processing Demo (No S3 required)
# Dataset: https://www.kaggle.com/datasets/mdhstama23/receipt-invoice-ml-ch2ps357
# Single Colab Cell — copy/paste & Run
# ===========================================

# ---------- CONFIG ----------
DATASET_SLUG = "mdhstama23/receipt-invoice-ml-ch2ps357"
OUT_DIR = "/content/receipt_demo_outputs"
MAX_IMAGES = 30           # limit for quick demo; set higher to process more
MAKE_PPTX = True          # export a 3-slide presentation with visuals
SEED = 13

# ---------- INSTALLS ----------
!pip -q install kaggle opencv-python-headless numpy pillow matplotlib scikit-image python-pptx tqdm

# ---------- IMPORTS ----------
import os, io, json, math, glob, zipfile, random, csv
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from skimage.filters import threshold_sauvola
from tqdm import tqdm
from datetime import datetime

random.seed(SEED)
np.random.seed(SEED)
os.makedirs(OUT_DIR, exist_ok=True)

# ---------- KAGGLE AUTH ----------
import pathlib, shutil
from google.colab import files

kaggle_dir = pathlib.Path("/root/.kaggle")
kaggle_dir.mkdir(parents=True, exist_ok=True)
kaggle_json = kaggle_dir / "kaggle.json"

if not kaggle_json.exists():
    print("Upload your kaggle.json (from https://www.kaggle.com/settings/account → Create New API Token)")
    uploaded = files.upload()
    if "kaggle.json" not in uploaded:
        raise RuntimeError("kaggle.json not uploaded. Please run the cell again and upload kaggle.json.")
    with open(kaggle_json, "wb") as f:
        f.write(uploaded["kaggle.json"])
    os.chmod(kaggle_json, 0o600)

# ---------- DOWNLOAD DATASET ----------
DATA_DIR = "/content/data"
os.makedirs(DATA_DIR, exist_ok=True)

print("Downloading dataset from Kaggle...")
!kaggle datasets download -d $DATASET_SLUG -p $DATA_DIR -o

# Unzip any downloaded zips
for z in glob.glob(os.path.join(DATA_DIR, "*.zip")):
    print("Unzipping:", os.path.basename(z))
    with zipfile.ZipFile(z, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)

# ---------- DISCOVER IMAGES ----------
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}
all_imgs = []
for root, _, files_in in os.walk(DATA_DIR):
    for fn in files_in:
        if os.path.splitext(fn.lower())[1] in IMG_EXTS:
            all_imgs.append(os.path.join(root, fn))

if not all_imgs:
    raise RuntimeError("No images found in the dataset. Inspect /content/data to verify contents.")

print(f"Found {len(all_imgs)} images. Sampling up to {MAX_IMAGES} for demo.")
demo_imgs = all_imgs if len(all_imgs) <= MAX_IMAGES else random.sample(all_imgs, MAX_IMAGES)

# ---------- IMAGE UTILS ----------
def imread_rgb(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        raise RuntimeError(f"Failed to read: {path}")
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def save_image(path, img):
    if img.ndim == 2:
        cv2.imwrite(path, img)
    else:
        cv2.imwrite(path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    return path

def ensure_uint8(img):
    if img.dtype == np.uint8:
        return img
    img = np.clip(img, 0, 255)
    return img.astype(np.uint8)

def show_side_by_side_and_save(a, b, titleA="Original", titleB="Pre-processed", save_path=None, dpi=140):
    plt.figure(figsize=(12,6), dpi=dpi)
    plt.subplot(1,2,1); plt.imshow(a); plt.axis('off'); plt.title(titleA)
    plt.subplot(1,2,2);
    if b.ndim == 2:
        plt.imshow(b, cmap='gray')
    else:
        plt.imshow(b)
    plt.axis('off'); plt.title(titleB)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    plt.show()

# ---------- RECEIPT-FOCUSED PREPROCESS ----------
def largest_receipt_quadrilateral(image_rgb):
    """Find the largest 4-point contour likely to be the receipt (or fallback to minAreaRect)."""
    image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(gray, 75, 200)

    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
    for c in contours:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            return approx.reshape(4,2)
    if len(contours) > 0:
        c = max(contours, key=cv2.contourArea)
        rect = cv2.minAreaRect(c)
        box = cv2.boxPoints(rect)
        return np.int0(box)
    return None

def order_points(pts):
    rect = np.zeros((4,2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]   # top-left
    rect[2] = pts[np.argmax(s)]   # bottom-right
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)] # top-right
    rect[3] = pts[np.argmax(diff)] # bottom-left
    return rect

def four_point_transform(image_rgb, pts):
    rect = order_points(pts.astype("float32"))
    (tl, tr, br, bl) = rect
    widthA  = np.linalg.norm(br - bl)
    widthB  = np.linalg.norm(tr - tl)
    heightA = np.linalg.norm(tr - br)
    heightB = np.linalg.norm(tl - bl)
    maxWidth  = int(max(widthA, widthB))
    maxHeight = int(max(heightA, heightB))
    dst = np.array([[0,0],[maxWidth-1,0],[maxWidth-1,maxHeight-1],[0,maxHeight-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR), M, (maxWidth, maxHeight), flags=cv2.INTER_CUBIC)
    return cv2.cvtColor(warped, cv2.COLOR_BGR2RGB)

def denoise_color(img_rgb):
    return cv2.fastNlMeansDenoisingColored(img_rgb, None, h=7, hColor=7, templateWindowSize=7, searchWindowSize=21)

def illumination_correction(gray):
    gray = ensure_uint8(gray)
    bg = cv2.morphologyEx(gray, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (35,35)))
    norm = cv2.divide(gray, bg, scale=255)
    return ensure_uint8(norm)

def clahe_enhance(gray):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    return clahe.apply(gray)

def unsharp(gray, amount=1.0, radius=3):
    blur = cv2.GaussianBlur(gray, (radius*2+1, radius*2+1), 0)
    sharp = cv2.addWeighted(gray, 1 + amount, blur, -amount, 0)
    return ensure_uint8(sharp)

def adaptive_binarize(gray, method="gaussian", block=25, C=15):
    gray = ensure_uint8(gray)
    if method == "sauvola":
        win = max(15, block)
        T = threshold_sauvola(gray, window_size=win, k=0.2, r=128)
        bw = (gray > T).astype(np.uint8) * 255
    else:
        adaptive_method = cv2.ADAPTIVE_THRESH_GAUSSIAN_C if method=="gaussian" else cv2.ADAPTIVE_THRESH_MEAN_C
        bw = cv2.adaptiveThreshold(gray, 255, adaptive_method, cv2.THRESH_BINARY, block, C)
    return bw

def morphology_clean(bw, open_ks=1, close_ks=1):
    if open_ks > 1:
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (open_ks, open_ks))
        bw = cv2.morphologyEx(bw, cv2.MORPH_OPEN, kernel)
    if close_ks > 1:
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (close_ks, close_ks))
        bw = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)
    return bw

# ---------- METRICS ----------
def tenengrad_sharpness(gray):
    gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    return np.mean(gx**2 + gy**2)

def edge_density(gray):
    e = cv2.Canny(gray, 100, 200)
    return e.sum() / 255.0 / e.size

def mser_text_regions(gray):
    mser = cv2.MSER_create(_delta=5, _min_area=60, _max_area=5000)
    regions, _ = mser.detectRegions(gray)
    return len(regions)

def composite_score(gray_or_bw):
    g = gray_or_bw if gray_or_bw.ndim==2 else cv2.cvtColor(gray_or_bw, cv2.COLOR_RGB2GRAY)
    g = ensure_uint8(g)
    s_sharp = tenengrad_sharpness(g)
    s_edge  = edge_density(g)
    s_mser  = mser_text_regions(g)
    score = 0.0
    score += (s_sharp / 1e4) * 0.5
    score += (s_edge * 5.0) * 1.0
    score += (min(s_mser, 300) / 300.0) * 1.5
    details = {"tenengrad": float(s_sharp), "edge_density": float(s_edge), "mser_regions": int(s_mser)}
    return float(score), details

# ---------- PIPELINE ----------
def preprocess_pipeline(img_rgb):
    # 1) Perspective correction
    quad = largest_receipt_quadrilateral(img_rgb)
    if quad is not None:
        flat = four_point_transform(img_rgb, quad)
        stage1 = flat
        stage1_note = "Auto perspective corrected"
    else:
        stage1 = img_rgb
        stage1_note = "Perspective correction skipped (no quadrilateral found)"

    # 2) Denoise + grayscale + illumination + CLAHE
    den   = denoise_color(stage1)
    gray0 = cv2.cvtColor(den, cv2.COLOR_RGB2GRAY)
    illum = illumination_correction(gray0)
    gclahe= clahe_enhance(illum)

    # 3) Variant sweep
    variants = []
    unsharp_amounts = [0.8, 1.2, 1.8]
    unsharp_radius  = [2, 3]
    methods = [("gaussian", 25, 15), ("mean", 25, 10), ("gaussian", 31, 12), ("sauvola", 25, 0)]
    morphs = [(1,1), (1,2), (2,2)]

    for a in unsharp_amounts:
        for r in unsharp_radius:
            g = unsharp(gclahe, amount=a, radius=r)
            for (m, block, C) in methods:
                bw = adaptive_binarize(g, method=m, block=block, C=C)
                for (op, cl) in morphs:
                    bw2 = morphology_clean(bw, open_ks=op, close_ks=cl)
                    score, details = composite_score(bw2)
                    variants.append({
                        "params": {"unsharp_amount": a, "unsharp_radius": r, "method": m, "block": block, "C": C, "open": op, "close": cl},
                        "image": bw2,
                        "score": score,
                        "details": details
                    })
    variants_sorted = sorted(variants, key=lambda d: d["score"], reverse=True)
    best = variants_sorted[0]
    return {
        "stage1_rgb": stage1,
        "stage1_note": stage1_note,
        "gray": gclahe,
        "best_bw": best["image"],
        "best_meta": best["params"],
        "best_score": best["score"],
        "best_details": best["details"],
        "topk": variants_sorted[:12]
    }

def grid_preview(variants, cols=4, tile_size=3.0, save_path=None):
    k = len(variants)
    rows = math.ceil(k / cols)
    plt.figure(figsize=(cols*tile_size, rows*tile_size))
    for i, v in enumerate(variants):
        plt.subplot(rows, cols, i+1)
        plt.imshow(v["image"], cmap='gray')
        p = v["params"]
        title = f"a={p['unsharp_amount']}, r={p['unsharp_radius']}\n{p['method']}({p['block']},{p['C']}) o{p['open']} c{p['close']}\nscore={v['score']:.2f}"
        plt.title(title, fontsize=8)
        plt.axis('off')
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=180)
    plt.show()

# ---------- RUN ON DEMO SET ----------
per_image_rows = []
before_after_paths = []

print(f"Processing {len(demo_imgs)} images...")
for idx, path in enumerate(tqdm(demo_imgs)):
    try:
        img = imread_rgb(path)
        res = preprocess_pipeline(img)

        # Save artifacts per image
        base = os.path.splitext(os.path.basename(path))[0]
        raw_out   = os.path.join(OUT_DIR, f"{idx:03d}_{base}_raw.jpg")
        flat_out  = os.path.join(OUT_DIR, f"{idx:03d}_{base}_flat.jpg")
        best_out  = os.path.join(OUT_DIR, f"{idx:03d}_{base}_best.png")
        side_out  = os.path.join(OUT_DIR, f"{idx:03d}_{base}_before_after.png")

        save_image(raw_out, img)
        save_image(flat_out, res["stage1_rgb"])
        save_image(best_out, res["best_bw"])
        show_side_by_side_and_save(img, res["best_bw"], save_path=side_out)
        before_after_paths.append(side_out)

        # Record metrics
        per_image_rows.append({
            "idx": idx,
            "filename": path,
            "stage1_note": res["stage1_note"],
            "best_score": round(res["best_score"], 4),
            "tenengrad": round(res["best_details"]["tenengrad"], 2),
            "edge_density": round(res["best_details"]["edge_density"], 4),
            "mser_regions": res["best_details"]["mser_regions"],
            "params": json.dumps(res["best_meta"])
        })

        # For the first image, also save a variant grid
        if idx == 0:
            grid_path = os.path.join(OUT_DIR, f"{idx:03d}_{base}_variant_grid_top12.png")
            grid_preview(res["topk"], save_path=grid_path)

    except Exception as e:
        print(f"[WARN] Failed {path}: {e}")

# ---------- SAVE METRICS CSV ----------
csv_path = os.path.join(OUT_DIR, "preprocessing_metrics.csv")
with open(csv_path, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=list(per_image_rows[0].keys()))
    writer.writeheader()
    for row in per_image_rows:
        writer.writerow(row)
print(f"\nMetrics CSV saved to: {csv_path}")

# ---------- PPTX (OPTIONAL) ----------
if MAKE_PPTX:
    from pptx import Presentation
    from pptx.util import Inches, Pt
    prs = Presentation()
    W, H = prs.slide_width, prs.slide_height

    # Slide 1: Title/Problem
    slide = prs.slides.add_slide(prs.slide_layouts[0])
    slide.shapes.title.text = "CR Model Enhancement: Fixing the Real Bottleneck"
    slide.placeholders[1].text = (
        "Skew, low contrast and shadows degrade YOLO/OCR accuracy.\n"
        "Solution: Fast, deterministic pre-processing (OpenCV) before inference."
    )

    # Slide 2: Before & After (use up to 2 examples)
    def add_image_slide(title, img_path):
        s = prs.slides.add_slide(prs.slide_layouts[5])
        tx = s.shapes.add_textbox(Inches(0.5), Inches(0.3), W - Inches(1), Inches(1))
        tx.text_frame.text = title
        tx.text_frame.paragraphs[0].font.size = Pt(28)
        s.shapes.add_picture(img_path, Inches(0.5), Inches(1.2), width=W - Inches(1))

    for i, p in enumerate(before_after_paths[:2]):
        add_image_slide(f"Before → After (Example {i+1})", p)

    # Slide 3: Integration & Scope
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    slide.shapes.title.text = "Integration & Next Steps"
    slide.placeholders[1].text = (
        "• Insert pre-processing before YOLO/OCR.\n"
        "• Handles skew, noise, shadows; outputs OCR-ready images.\n"
        "• Monitored via text-likeness metrics.\n\n"
        "Budget Fit: This micro-milestone fits the $250 scope.\n"
        "Optionally follow with targeted model updates."
    )

    pptx_path = os.path.join(OUT_DIR, "CR_Model_Enhancement_Demo.pptx")
    prs.save(pptx_path)
    print(f"PPTX saved to: {pptx_path}")

print("\n=== DONE ===")
print(f"Artifacts directory: {OUT_DIR}")
print("What to show the client:")
print("1) A few of the *_before_after.png images (clear improvement).")
print("2) The metrics CSV showing per-image text-likeness gains.")
print("3) The PPTX for a 2-minute walkthrough.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 8.4 MB/s eta 0:00:00
Upload your kaggle.json (from https://www.kaggle.com/settings/account → Create New API Token)
